# Sentiment Analysis Exploration

In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns          
import re 
import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.sequence import pad_sequences

In [87]:
data = pd.read_csv('data/sentiment-analysis.csv')
data.head()

,"Text, Sentiment, Source, Date/Time, User ID, Location, Confidence Score"
0,"""I love this product!"", Positive, Twitter, 202..."
1,"""The service was terrible."", Negative, Yelp Re..."
2,"""This movie is amazing!"", Positive, IMDb, 2023..."
3,"""I'm so disappointed with their customer suppo..."
4,"""Just had the best meal of my life!"", Positive..."


In [88]:
display(data.isna().value_counts())
data = data.dropna()

Text, Sentiment, Source, Date/Time, User ID, Location, Confidence Score
False                                                                      96
True                                                                        2
Name: count, dtype: int64

In [89]:
column_names = [header.replace(' ', '') for header in data.columns[0].split(',')]
column_names

['Text',
 'Sentiment',
 'Source',
 'Date/Time',
 'UserID',
 'Location',
 'ConfidenceScore']

In [90]:
df = pd.DataFrame(columns=column_names)
df

,Text,Sentiment,Source,Date/Time,UserID,Location,ConfidenceScore


In [91]:
data

,"Text, Sentiment, Source, Date/Time, User ID, Location, Confidence Score"
0,"""I love this product!"", Positive, Twitter, 202..."
1,"""The service was terrible."", Negative, Yelp Re..."
2,"""This movie is amazing!"", Positive, IMDb, 2023..."
3,"""I'm so disappointed with their customer suppo..."
4,"""Just had the best meal of my life!"", Positive..."
...,...
91,"""Just had the most amazing vacation! I can't w..."
92,"""The food at this restaurant was awful. Never ..."
93,"""I can't stop listening to this song. It's my ..."
94,"""Their website is so confusing and poorly desi..."


In [92]:
row_values = [row.iloc[0].split(',') for _, row in data.iterrows()]

In [95]:
for row in range(0, len(row_values)):
    for ind, col_name in enumerate(column_names):
        df.at[row, col_name] = row_values[row][ind]

In [99]:
df

,Text,Sentiment,Source,Date/Time,UserID,Location,ConfidenceScore
0,"""I love this product!""",Positive,Twitter,2023-06-15 09:23:14,@user123,New York,0.85
1,"""The service was terrible.""",Negative,Yelp Reviews,2023-06-15 11:45:32,user456,Los Angeles,0.65
2,"""This movie is amazing!""",Positive,IMDb,2023-06-15 14:10:22,moviefan789,London,0.92
3,"""I'm so disappointed with their customer suppo...",Negative,Online Forum,2023-06-15 17:35:11,forumuser1,Toronto,0.78
4,"""Just had the best meal of my life!""",Positive,TripAdvisor,2023-06-16 08:50:59,foodie22,Paris,0.88
...,...,...,...,...,...,...,...
91,"""Just had the most amazing vacation! I can't w...",Positive,TripAdvisor,2023-07-02 18:01:23,travelenthusiast1,Sydney,0.93
92,"""The food at this restaurant was awful. Never ...",Negative,Zomato,2023-07-02 20:45:37,foodlover123,Mumbai,0.55
93,"""I can't stop listening to this song. It's my ...",Positive,Spotify,2023-07-03 09:17:52,musiclover789,Berlin,0.91
94,"""Their website is so confusing and poorly desi...",Negative,Website Review,2023-07-03 11:59:18,user789,Toronto,0.68
